In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, f1_score, accuracy_score
import tensorflow.keras as keras
import time
import numpy as np
import matplotlib.pyplot as plt
import math
from deep_audio import Directory, JSON
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
import joblib

In [ ]:
method_algo = 'mfcc'
model_algo = 'perceptron'
n_rate = 24000

DATASET_PATH = f'processed/{method_algo}/{method_algo}_{n_rate}.json'

inputs, targets, mapping = Directory.load_json_data(DATASET_PATH, inputs_fieldname=method_algo)

In [ ]:
def build_model():
    # build the network architecture
    model = keras.Sequential([
        # input layer
        keras.layers.Flatten(input_shape=(inputs.shape[1], inputs.shape[2])),

        # 1st hidden layer
        keras.layers.Dense(512, activation='relu'),

        # 2nd hidden layer
        keras.layers.Dense(256, activation='relu'),

        # 3rd hidden layer
        keras.layers.Dense(128, activation='relu'),

        # output layer
        keras.layers.Dense(len(mapping), activation='softmax'),
    ])
    
    optimizer = keras.optimizers.Adam(learning_rate=0.0001)

    model.compile(optimizer=optimizer,
          loss='sparse_categorical_crossentropy',
          metrics=['accuracy'])

    return model

In [ ]:
# SPLIT DOS DADOS
random_state = 42


X_train, X_test, y_train, y_test = train_test_split(inputs,
                                                                          targets,
                                                                          test_size=0.2,
                                                                          stratify=targets,
                                                                          random_state=random_state)

X_train, X_valid, y_train, y_valid = train_test_split(X_train,
                                                                          y_train,
                                                                          test_size=0.2,
                                                                          stratify=y_train,
                                                                          random_state=random_state)


In [ ]:
param_grid = {}

In [ ]:
model = KerasClassifier(build_fn=build_model, epochs=2000, batch_size=128, verbose=1, )

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=5)

grid_result = grid.fit(inputs_train, targets_train, validation_data=(inputs_valid, targets_valid))

In [ ]:
# # print best parameter after tuning
model = grid
best_params = model.best_params_
sampling_rate = n_rate
# TESTA ACCURÁCIAS

score_test = model.score(X_test, y_test)
score_valid = model.score(X_valid, y_valid)
score_train = model.score(X_train, y_train)

y_hat = model.predict(X_test)

# SALVA MODELO
# filename = f'models/gridperceptron/{method_algo}_{f1_score(y_hat, y_test, average="macro")}_{sampling_rate}/acc{score_test}_seed{random_state}.sav'

# Directory.create_directory(filename, file=True)

# joblib.dump(model, filename)

# SALVA ACURÁCIAS E PARAMETROS
dump_info = {
    'method': 'Grid Search Random Forest',
    'seed': random_state,
    'feature_method': method_algo,
    'sample_rate': sampling_rate,
    'train_test': [len(X_train), len(X_valid), len(X_test)],
    'score_train': score_train,
    'score_valid': score_valid,
    'score_test': score_test,
    'f1_micro': f1_score(y_hat, y_test, average='micro'),
    'f1_macro': f1_score(y_hat, y_test, average='macro'),
    'model_file': f'acc{score_test}_seed{random_state}.sav',
    'params': model.best_params_,
    'cv_results': model.cv_results_
}

JSON.create_json_file(f'models/gridperceptron/{method_algo}_{f1_score(y_hat, y_test, average="macro")}_{sampling_rate}/info.json', dump_info)

In [ ]:
# SALVA A ESTRUTURA DO MODELO

# timestamp = int(time.time())

# Directory.create_directory(f'models/{model_algo}/{method_algo}/{timestamp}')

# JSON.create_json_file(f'models/{model_algo}/{method_algo}/{timestamp}/model_structure.json', model.to_json())

# model_save_filename = f'models/{model_algo}/{method_algo}/{timestamp}/model_weight.h5'

# # DECIDE QUANDO PARAR
# earlystopping_cb = keras.callbacks.EarlyStopping(patience=300, restore_best_weights=True)

# # SALVA OS PESOS
# mdlcheckpoint_cb = keras.callbacks.ModelCheckpoint(
# model_save_filename, monitor="val_accuracy", save_best_only=True
# )

In [ ]:
# TREINA O MODELO
# history = model.fit(inputs_train, targets_train,
#             validation_data=(inputs_valid, targets_valid),
#             epochs=10000,
#             batch_size=128,
#             callbacks=[earlystopping_cb, mdlcheckpoint_cb])

In [ ]:
# GERA O GRAFICO DE ACURÁCIA
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.savefig(f'models/{model_algo}/{method_algo}/{timestamp}/graph_accuracy.png')
plt.close()

# GERA O GRÁFICO DE PERCA
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.savefig(f'models/{model_algo}/{method_algo}/{timestamp}/graph_loss.png')
plt.close()

In [ ]:
# PEGA A MAIOR E ACURÁCIA
higher_accuracy = model.evaluate(inputs_test, targets_test, batch_size=128)

higher_accuracy = str(int(higher_accuracy[1] * 10000)).zfill(4)

In [ ]:
# RENOMEIA A PASTA
Directory.rename_directory(f'models/{model_algo}/{method_algo}/{timestamp}',
                   f'models/{model_algo}/{method_algo}/acc{higher_accuracy}_seed{random_state}_epochs{len(history.history["accuracy"])}_time{timestamp}')